In [ ]:
"""
For assignment6, I got a speedup of 54%.
"""
 
""" 
    
    The relative speedups are: 
    {1: 2.5943132204285995, 2: 1.162570218357162, 3: 1.0218080537714478, 4: 0.99983733795216001, 'orig': 1.0, 'opt': 2.8114968869479546}
    
    The time it took to run is:
    {1: 30.994290845002979, 2: 69.164767191978171, 3: 78.692762500955723, 4: 80.421980100974906, 'orig': 80.408898497000337, 'opt': 28.600031132984441}
    
    I did break 30 seconds. :)
    
"""

"""
    By far, the biggest change was removing the duplicated function calls.
    
    I tried parallelization for the loops where it was safe to do them, but it was actually slower.     
     
    N-body simulation.
"""
import itertools
import numpy as np

PI = 3.14159265358979323
SOLAR_MASS = 4 * PI * PI
DAYS_PER_YEAR = 365.24

BODIES = {
    'sun': (np.array([0.0, 0.0, 0.0]), np.array([0.0, 0.0, 0.0]), SOLAR_MASS),

    'jupiter': (np.array([4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01]),
                np.array([1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR]),
                9.54791938424326609e-04 * SOLAR_MASS),

    'saturn': (np.array([8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01]),
               np.array([-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR]),
               2.85885980666130812e-04 * SOLAR_MASS),

    'uranus': (np.array([1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01]),
               np.array([2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR]),
               4.36624404335156298e-05 * SOLAR_MASS),

    'neptune': (np.array([1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01]),
                np.array([2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR]),
                5.15138902046611451e-05 * SOLAR_MASS)}

VISIT_SCHEDULE = list(itertools.combinations(BODIES.keys(), 2))
BODIES1, BODIES2= zip(*VISIT_SCHEDULE)
BODIES_KEYS = list(BODIES.keys())
dt = 0.01

def to_do(body1, body2, BODIES=BODIES, dt=dt):
    (xyz1, v1, m1) = BODIES[body1]
    (xyz2, v2, m2) = BODIES[body2]
    d = xyz1-xyz2
    
    mag = dt * (np.inner(d,d) ** (-1.5))

    body1_velocity_arg = m2 * mag
    v1 = d * body1_velocity_arg


    body2_velocity_arg = m1 * mag       
    v2 = d * body1_velocity_arg


def update_rs_for_body(body, BODIES=BODIES, dt=dt):
    (r, vxyz, m) = BODIES[body]
    r = dt * vxyz

    
def report_energy(e=0.0, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS):
    '''
        compute the energy and return it so that it can be printed
    '''
    for (body1, body2) in VISIT_SCHEDULE:
        (xyz1, v1, m1) = BODIES[body1]
        (xyz2, v2, m2) = BODIES[body2]

        d = xyz1 - xyz2

        # Amazingly the code as written is faster than this code below.
        #e -= (m1 * m2) / np.inner(d,d)**0.5
        e -= (m1 * m2) / (d[0]**2 + d[1]**2 + d[2]**2)**0.5

        
    for body in BODIES_KEYS:
        (r, v, m) = BODIES[body]
        e += m * np.inner(v,v) / 2.
        
    return e

def offset_momentum(ref, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    p = np.zeros(3)
    for body in BODIES_KEYS:
        (r, v, m) = BODIES[body]
        p -= v*m
        
    (r, v, m) = ref
    v = p / m


def nbody(loops, reference, iterations, VISIT_SCHEDULE=VISIT_SCHEDULE, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS, BODIES1=BODIES1, BODIES2=BODIES2):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # Set up global state
    offset_momentum(BODIES[reference])

    total_runs = loops * iterations
    for i, (body1, body2) in enumerate(itertools.cycle(VISIT_SCHEDULE)):
        to_do(body1, body2)
        
        if (i % iterations) == 0:
            for body in BODIES_KEYS:
                update_rs_for_body(body)
                
        if (i % loops) == 0:
            print(report_energy())
            
        if i >= total_runs - 1:
            break
      


if __name__ == '__main__':
    nbody(100, 'sun', 20000)


In [ ]:
'''
    nbody simulation
    loops - number of loops to run
    reference - body at center of system
    iterations - number of timesteps to advance
'''
# initialize stuff
PI = 3.14159265358979323
SOLAR_MASS = 4 * PI * PI
DAYS_PER_YEAR = 365.24
BODIES_NAMES = ['sun', 'jupiter', 'saturn', 'uranus', 'neptune']
KEYS = np.arange(5)
POSITIONS = np.array([
    [0.0, 0.0, 0.0],
    [4.84143144246472090e+00,                 -1.16032004402742839e+00,                 -1.03622044471123109e-01],
    [8.34336671824457987e+00,                4.12479856412430479e+00,                -4.03523417114321381e-01],
    [1.28943695621391310e+01,                -1.51111514016986312e+01,                -2.23307578892655734e-01],
    [1.53796971148509165e+01,                 -2.59193146099879641e+01,                 1.79258772950371181e-01],
])

VELOCITIES = np.array([
    [0.0, 0.0, 0.0],
    [1.66007664274403694e-03 * DAYS_PER_YEAR,                 7.69901118419740425e-03 * DAYS_PER_YEAR,                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
    [-2.76742510726862411e-03 * DAYS_PER_YEAR,                4.99852801234917238e-03 * DAYS_PER_YEAR,                2.30417297573763929e-05 * DAYS_PER_YEAR],
    [2.96460137564761618e-03 * DAYS_PER_YEAR,                2.37847173959480950e-03 * DAYS_PER_YEAR,                -2.96589568540237556e-05 * DAYS_PER_YEAR],
    [2.68067772490389322e-03 * DAYS_PER_YEAR,                 1.62824170038242295e-03 * DAYS_PER_YEAR,                 -9.51592254519715870e-05 * DAYS_PER_YEAR],  
])

MASSES = np.array([ SOLAR_MASS, 9.54791938424326609e-04 * SOLAR_MASS, 2.85885980666130812e-04 * SOLAR_MASS, 4.36624404335156298e-05 * SOLAR_MASS, 5.15138902046611451e-05 * SOLAR_MASS,])
VISIT_SCHEDULE = np.array(list(combinations(KEYS, 2)))
dt = 0.01
# ================

# Cleaned Up Version

In [2]:
#%%file nbody_cleaned_up_with_numpy.py
import itertools
import numpy as np

# initialize stuff
PI = 3.14159265358979323
SOLAR_MASS = 4 * PI * PI
DAYS_PER_YEAR = 365.24
BODIES_NAMES = ['sun', 'jupiter', 'saturn', 'uranus', 'neptune']
BODIES_KEYS = np.arange(5)
POSITIONS = np.array([
    [0.0, 0.0, 0.0],
    [4.84143144246472090e+00,                 -1.16032004402742839e+00,                 -1.03622044471123109e-01],
    [8.34336671824457987e+00,                4.12479856412430479e+00,                -4.03523417114321381e-01],
    [1.28943695621391310e+01,                -1.51111514016986312e+01,                -2.23307578892655734e-01],
    [1.53796971148509165e+01,                 -2.59193146099879641e+01,                 1.79258772950371181e-01],
])

VELOCITIES = np.array([
    [0.0, 0.0, 0.0],
    [1.66007664274403694e-03 * DAYS_PER_YEAR,                 7.69901118419740425e-03 * DAYS_PER_YEAR,                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
    [-2.76742510726862411e-03 * DAYS_PER_YEAR,                4.99852801234917238e-03 * DAYS_PER_YEAR,                2.30417297573763929e-05 * DAYS_PER_YEAR],
    [2.96460137564761618e-03 * DAYS_PER_YEAR,                2.37847173959480950e-03 * DAYS_PER_YEAR,                -2.96589568540237556e-05 * DAYS_PER_YEAR],
    [2.68067772490389322e-03 * DAYS_PER_YEAR,                 1.62824170038242295e-03 * DAYS_PER_YEAR,                 -9.51592254519715870e-05 * DAYS_PER_YEAR],  
])

MASSES = np.array([ SOLAR_MASS, 9.54791938424326609e-04 * SOLAR_MASS, 2.85885980666130812e-04 * SOLAR_MASS, 4.36624404335156298e-05 * SOLAR_MASS, 5.15138902046611451e-05 * SOLAR_MASS,])
VISIT_SCHEDULE = np.array(list(itertools.combinations(BODIES_KEYS, 2)))
dt = 0.01
# ================


def to_do(body1, body2, POSITIONS, VELOCITIES, MASSES, dt):
    (xyz1, v1, m1) = POSITIONS[body1], VELOCITIES[body1], MASSES[body1]
    (xyz2, v2, m2) = POSITIONS[body2], VELOCITIES[body2], MASSES[body2]
    d = xyz1-xyz2
    
    mag = dt * (np.inner(d,d) ** (-1.5))

    body1_velocity_arg = m2 * mag
    v1[:] -= d * body1_velocity_arg


    body2_velocity_arg = m1 * mag       
    v2[:] += d * body1_velocity_arg


def update_rs_for_body(body, POSITIONS, VELOCITIES, MASSES, dt):
    (r, vxyz, m) = POSITIONS[body], VELOCITIES[body], MASSES[body]
    r[:] += dt * vxyz

    
def report_energy(POSITIONS, VELOCITIES, MASSES, VISIT_SCHEDULE, BODIES_KEYS):
    '''
        compute the energy and return it so that it can be printed
    '''
    e = 0.0
    for (body1, body2) in VISIT_SCHEDULE:
        (xyz1, v1, m1) = POSITIONS[body1], VELOCITIES[body1], MASSES[body1]
        (xyz2, v2, m2) = POSITIONS[body2], VELOCITIES[body2], MASSES[body2]

        d = xyz1 - xyz2

        # Amazingly the code as written is faster than this code below.
        #e -= (m1 * m2) / np.inner(d,d)**0.5
        e -= (m1 * m2) / (d[0]**2 + d[1]**2 + d[2]**2)**0.5

        
    for body in BODIES_KEYS:
        (r, v, m) = POSITIONS[body], VELOCITIES[body], MASSES[body]
        e += m * np.inner(v,v) / 2.
        
    return e

def offset_momentum(ref, POSITIONS, VELOCITIES, MASSES, BODIES_KEYS):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    p = np.zeros(3)
    for body in BODIES_KEYS:
        (r, v, m) = POSITIONS[body], VELOCITIES[body], MASSES[body]
        p -= v*m
        
    (r, v, m) = POSITIONS[ref], VELOCITIES[ref], MASSES[ref]
    
    # Necessary syntax to allow memory sharing
    v[:] = p / m


def nbody(loops, reference, iterations,
          POSITIONS=POSITIONS, VELOCITIES=VELOCITIES, MASSES=MASSES,
          VISIT_SCHEDULE=VISIT_SCHEDULE, BODIES_KEYS=BODIES_KEYS
         ):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # Set up global state
    offset_momentum(0, POSITIONS, VELOCITIES, MASSES, BODIES_KEYS)

    total_runs = loops * iterations
    for i, (body1, body2) in enumerate(itertools.cycle(VISIT_SCHEDULE)):
        to_do(body1, body2, POSITIONS, VELOCITIES, MASSES, dt)
        
        if (i % iterations) == 0:
            for body in BODIES_KEYS:
                update_rs_for_body(body, POSITIONS, VELOCITIES, MASSES, dt)
                
        if (i % loops) == 0:
            print(report_energy(POSITIONS, VELOCITIES, MASSES, VISIT_SCHEDULE, BODIES_KEYS))
            
        if i >= total_runs - 1:
            break
      


if __name__ == '__main__':
    nbody(100, 'sun', 20000)


-0.169114178632
-0.169109483799
-0.169103742237
-0.169096953947
-0.169089118929
-0.169080237182
-0.169070308707
-0.169059333504
-0.169047311573
-0.169034242913
-0.169020127525
-0.169004965409
-0.168988756564
-0.168971500991
-0.16895319869
-0.16893384966
-0.168913453902
-0.168892011416
-0.168869522201
-0.168845986258
-0.168821403587
-0.168795774188
-0.16876909806
-0.168741375204
-0.16871260562
-0.168682789307
-0.168651926266
-0.168620016497
-0.168587059999
-0.168553056774
-0.168518006819
-0.168481910137
-0.168444766726
-0.168406576587
-0.16836733972
-0.168327056124
-0.1682857258
-0.168243348748
-0.168199924967
-0.168155454458
-0.168109937221
-0.168063373256
-0.168015762562
-0.16796710514
-0.167917400989
-0.167866650111
-0.167814852504
-0.167762008168
-0.167708117105
-0.167653179313
-0.167597194793
-0.167540163544
-0.167482085567
-0.167422960862
-0.167362789429
-0.167301571267
-0.167239306377
-0.167175994759
-0.167111636412
-0.167046231337
-0.166979779534
-0.166912281002
-0.166843735742


KeyboardInterrupt: 

# Assignment 8 Scratchwork

In [38]:
%%file nbody_numba.py
import itertools
import numpy as np
from numba import jit, int32, float64, char, vectorize

@vectorize([float64(float64, float64)])
def vec_deltas(a, b):
    # fast computation of deltas, as per assignment
    return a - b

@jit("float64(float64[:])")
def l2_norm_len3(a):
    # computes the L2 norm very very quickly
    return a[0] * a[0] + a[1] * a[1] + a[2] * a[2]

@jit("void(int32, int32, float64[:,:], float64[:,:], float64[:], float64)", nopython=True)
def to_do(body1, body2, POSITIONS, VELOCITIES, MASSES, dt):
    (xyz1, v1, m1) = POSITIONS[body1], VELOCITIES[body1], MASSES[body1]
    (xyz2, v2, m2) = POSITIONS[body2], VELOCITIES[body2], MASSES[body2]

    d = vec_deltas(xyz1, xyz2)
    
    mag = dt * (l2_norm_len3(d) ** (-1.5))

    

    body1_velocity_arg = m2 * mag
    v1[:] -= d * body1_velocity_arg


    body2_velocity_arg = m1 * mag       
    v2[:] += d * body1_velocity_arg

@jit("void(int32, float64[:,:], float64[:,:], float64[:], float64)", nopython=True)
def update_rs_for_body(body, POSITIONS, VELOCITIES, MASSES, dt):
    (r, vxyz, m) = POSITIONS[body], VELOCITIES[body], MASSES[body]
    r[:] += dt * vxyz

@jit("float64(float64[:,:], float64[:,:], float64[:], int32[:])", nopython=True)
def report_energy(POSITIONS, VELOCITIES, MASSES, BODIES_KEYS):
    '''
        compute the energy and return it so that it can be printed
    '''
    e = float(0.0)
    
    for body1 in BODIES_KEYS:
        for body2 in range(body1+1, len(BODIES_KEYS)):

            #(body1, body2) = VISIT_SCHEDULE[i, 0], VISIT_SCHEDULE[i, 1]
            (xyz1, v1, m1) = POSITIONS[body1], VELOCITIES[body1], MASSES[body1]
            (xyz2, v2, m2) = POSITIONS[body2], VELOCITIES[body2], MASSES[body2]

            d = vec_deltas(xyz1, xyz2)
            
            # Amazingly the code as written is faster than this code below.
            e -= (m1 * m2) / np.sqrt(l2_norm_len3(d))

        
    for body in BODIES_KEYS:
        (r, v, m) = POSITIONS[body], VELOCITIES[body], MASSES[body]
        e += m * (l2_norm_len3(v)) / 2.
        
    return e

@jit("void(int32, float64[:,:], float64[:,:], float64[:], int32[:])", nopython=True)
def offset_momentum(ref, POSITIONS, VELOCITIES, MASSES, BODIES_KEYS):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    p = np.zeros(3)
    for body in BODIES_KEYS:
        (r, v, m) = POSITIONS[body], VELOCITIES[body], MASSES[body]
        p -= v*m
        
    (r, v, m) = POSITIONS[ref], VELOCITIES[ref], MASSES[ref]
    
    # Necessary syntax to allow memory sharing
    v[:] = p / m

@jit("void(int32, char, int32)")
def nbody(loops, reference, iterations):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    
    # initialize stuff
    # initialization done inside for the sake of jit and shared-memory issues
    # ====================================
    PI = 3.14159265358979323
    SOLAR_MASS = 4 * PI * PI
    DAYS_PER_YEAR = 365.24
    BODIES_NAMES = ['sun', 'jupiter', 'saturn', 'uranus', 'neptune']
    BODIES_KEYS = np.array( list(range(5)), dtype=np.int32)
    POSITIONS = np.array([
        [0.0, 0.0, 0.0],
        [4.84143144246472090e+00,                 -1.16032004402742839e+00,                 -1.03622044471123109e-01],
        [8.34336671824457987e+00,                4.12479856412430479e+00,                -4.03523417114321381e-01],
        [1.28943695621391310e+01,                -1.51111514016986312e+01,                -2.23307578892655734e-01],
        [1.53796971148509165e+01,                 -2.59193146099879641e+01,                 1.79258772950371181e-01],
    ], dtype=np.float64)

    VELOCITIES = np.array([
        [0.0, 0.0, 0.0],
        [1.66007664274403694e-03 * DAYS_PER_YEAR,                 7.69901118419740425e-03 * DAYS_PER_YEAR,                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
        [-2.76742510726862411e-03 * DAYS_PER_YEAR,                4.99852801234917238e-03 * DAYS_PER_YEAR,                2.30417297573763929e-05 * DAYS_PER_YEAR],
        [2.96460137564761618e-03 * DAYS_PER_YEAR,                2.37847173959480950e-03 * DAYS_PER_YEAR,                -2.96589568540237556e-05 * DAYS_PER_YEAR],
        [2.68067772490389322e-03 * DAYS_PER_YEAR,                 1.62824170038242295e-03 * DAYS_PER_YEAR,                 -9.51592254519715870e-05 * DAYS_PER_YEAR],  
    ], dtype=np.float64)
    VISIT_SCHEDULE = np.array(list(itertools.combinations(BODIES_KEYS, 2)), dtype=np.int32)

    MASSES = np.array([ SOLAR_MASS, 9.54791938424326609e-04 * SOLAR_MASS, 2.85885980666130812e-04 * SOLAR_MASS, 4.36624404335156298e-05 * SOLAR_MASS, 5.15138902046611451e-05 * SOLAR_MASS,])
    dt = 0.01
    # ====================================
    
    
    
    
    
    # Set up global state
    offset_momentum(BODIES_NAMES.index(reference), POSITIONS, VELOCITIES, MASSES, BODIES_KEYS)

    total_runs = loops * iterations
    for i, (body1, body2) in enumerate(itertools.cycle(VISIT_SCHEDULE)):
        to_do(body1, body2, POSITIONS, VELOCITIES, MASSES, dt)
    
        
        if (i % iterations) == 0:
            for body in BODIES_KEYS:
                update_rs_for_body(body, POSITIONS, VELOCITIES, MASSES, dt)
                
        if (i % loops) == 0:
            print(report_energy(POSITIONS, VELOCITIES, MASSES, BODIES_KEYS))
                  


if __name__ == '__main__':
    nbody(100, 'sun', 20000)


Overwriting nbody_numba.py


In [65]:
import numba
list(map(numba.typeof, [POSITIONS, VELOCITIES, MASSES, VISIT_SCHEDULE, BODIES_KEYS]))

[array(float64, 2d, C),
 array(float64, 2d, C),
 array(float64, 1d, C),
 array(int32, 2d, C),
 array(int32, 1d, C)]

In [85]:
numba.typeinfer

<module 'numba.typeinfer' from '/Users/akunas/anaconda/envs/py35/lib/python3.5/site-packages/numba/typeinfer.py'>

In [22]:
@jit("float64(float64[:])")
def l2_norm(a):
    return np.sum(a*a)

@jit("float64(float64[:])")
def l2_norm2(a):
    return a.dot(a)

@jit("float64(float64[:])")
def l2_norm3(a):
    return a[0] * a[0] + a[1] * a[1] + a[2] * a[2]

In [18]:
a = np.array([1,2,3], dtype=np.float64)
b = np.array([4,5,6], dtype=np.float64)

l2_norm(a, b)
l2_norm2(a, b)

32.0

In [23]:
%timeit l2_norm(a)


The slowest run took 20.03 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 456 ns per loop


In [24]:
%timeit l2_norm2(a)

The slowest run took 15.45 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.86 µs per loop


In [25]:
%timeit l2_norm3(a)

The slowest run took 20.79 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 272 ns per loop


In [33]:
@jit("float64(float64)")
def sq1(a):
    return a**1.5
@jit("float64(float64)")
def sq2(a):
    return a*np.sqrt(a)

In [34]:
%timeit sq1(10)

The slowest run took 24.21 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 207 ns per loop


In [35]:
%timeit sq2(10)

The slowest run took 35.16 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 186 ns per loop


In [3]:
import numpy as np
import itertools
PI = 3.14159265358979323
SOLAR_MASS = 4 * PI * PI
DAYS_PER_YEAR = 365.24
#BODIES_NAMES = ['sun', 'jupiter', 'saturn', 'uranus', 'neptune']
BODIES_KEYS = np.array( list(range(5)), dtype=np.int32)
POSITIONS = np.array([
    [0.0, 0.0, 0.0],
    [4.84143144246472090e+00,                 -1.16032004402742839e+00,                 -1.03622044471123109e-01],
    [8.34336671824457987e+00,                4.12479856412430479e+00,                -4.03523417114321381e-01],
    [1.28943695621391310e+01,                -1.51111514016986312e+01,                -2.23307578892655734e-01],
    [1.53796971148509165e+01,                 -2.59193146099879641e+01,                 1.79258772950371181e-01],
], dtype=np.float64)

VELOCITIES = np.array([
    [0.0, 0.0, 0.0],
    [1.66007664274403694e-03 * DAYS_PER_YEAR,                 7.69901118419740425e-03 * DAYS_PER_YEAR,                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
    [-2.76742510726862411e-03 * DAYS_PER_YEAR,                4.99852801234917238e-03 * DAYS_PER_YEAR,                2.30417297573763929e-05 * DAYS_PER_YEAR],
    [2.96460137564761618e-03 * DAYS_PER_YEAR,                2.37847173959480950e-03 * DAYS_PER_YEAR,                -2.96589568540237556e-05 * DAYS_PER_YEAR],
    [2.68067772490389322e-03 * DAYS_PER_YEAR,                 1.62824170038242295e-03 * DAYS_PER_YEAR,                 -9.51592254519715870e-05 * DAYS_PER_YEAR],  
], dtype=np.float64)
VISIT_SCHEDULE = np.array(list(itertools.combinations(BODIES_KEYS, 2)), dtype=np.int32)

MASSES = np.array([ SOLAR_MASS, 9.54791938424326609e-04 * SOLAR_MASS, 2.85885980666130812e-04 * SOLAR_MASS, 4.36624404335156298e-05 * SOLAR_MASS, 5.15138902046611451e-05 * SOLAR_MASS,])
dt = 0.01

In [4]:
VISIT_SCHEDULE

array([[0, 1],
       [0, 2],
       [0, 3],
       [0, 4],
       [1, 2],
       [1, 3],
       [1, 4],
       [2, 3],
       [2, 4],
       [3, 4]], dtype=int32)